## 필요 라이브러리

In [48]:
import os
import numpy as np
import pandas as pd
import mne
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from glob import glob

## 필요 함수 정의

### SuS Epoching

In [49]:
def epoch_sus_trials(
    eeg_file,
    events_file,
    tmin=-0.2,
    tmax=0.5,
    stim_label='stim_ON',
    baseline=None,                 # None이면 MNE 기본, 기존 코드와 비슷하게 (tmin, 0)을 원하면 baseline=(tmin, 0)
    detrend=1,
    picks=None,                    # 예: picks = mne.pick_types(raw.info, eeg=True)
    reject_threshold=150e-6,       # EEG peak-to-peak threshold (Volts). None이면 reject 사용 안함.
    group_size=None,               # int: 그룹 평균 크기 (None이면 그룹화 안함)
    allow_overlap=False,           # 그룹 샘플링 시 replacement 허용 여부
    random_state=0,
    return_original_epochs=False   # True면 (epochs_array, grouped_array, subj), False면 (X_or_grouped, subj)
):
    """
    Safer SuS epoching helper.
    - 기본 반환: (X, subj)  where X is epochs (n_trials, n_channels, n_times) if group_size is None.
    - 만약 group_size가 주어지면 grouped ERPs 반환: (n_groups, n_channels, n_times).
    - return_original_epochs=True 로 하면 원본 epochs도 추가 반환.
    """
    rng = np.random.default_rng(random_state)

    # 1) read raw
    raw = mne.io.read_raw_eeglab(eeg_file, preload=True, verbose=False)

    # 2) read events table (robust)
    events_df = pd.read_csv(events_file, sep="\t")
    # try several column names for the value/type and onset
    if 'value' in events_df.columns:
        val_col = 'value'
    elif 'type' in events_df.columns:
        val_col = 'type'
    else:
        # fallback: take first non-onset column
        cols = [c for c in events_df.columns if c.lower() not in ('onset','time','latency','sample')]
        val_col = cols[0] if cols else events_df.columns[0]

    if 'onset' in events_df.columns:
        onset_col = 'onset'
    elif 'time' in events_df.columns:
        onset_col = 'time'
    elif 'latency' in events_df.columns:
        onset_col = 'latency'
    else:
        raise ValueError("events_file에 onset/time/latency 컬럼이 필요합니다.")

    # select stim events (case-insensitive)
    mask = events_df[val_col].astype(str).str.lower() == stim_label.lower()
    stim_events = events_df[mask].reset_index(drop=True)
    if stim_events.empty:
        # try contains (fallback)
        mask = events_df[val_col].astype(str).str.lower().str.contains(stim_label.lower())
        stim_events = events_df[mask].reset_index(drop=True)
        if stim_events.empty:
            raise ValueError(f"No events named '{stim_label}' found in {events_file} (cols: {val_col}).")

    # 3) convert onset units to seconds robustly
    onsets = stim_events[onset_col].astype(float).values
    rec_dur_sec = raw.times[-1]
    sfreq = float(raw.info['sfreq'])

    # heuristics:
    # - if max(onsets) >> recording duration, it might be ms or samples.
    if onsets.max() > rec_dur_sec * 1.5:
        # likely ms (divide by 1000) or samples (divide by sfreq)
        if onsets.max() > 1e4:  # large numbers -> probably ms
            onsets_sec = onsets / 1000.0
        elif onsets.max() > raw.n_times:  # larger than number of samples -> probably ms
            onsets_sec = onsets / 1000.0
        else:
            # as a fallback, treat as samples
            onsets_sec = onsets / sfreq
    else:
        onsets_sec = onsets

    # ensure sample indices valid
    onset_samples = (onsets_sec * sfreq).astype(int)
    onset_samples = np.clip(onset_samples, 0, raw.n_times - 1)

    # build MNE events array
    mne_events = np.column_stack([onset_samples, np.zeros_like(onset_samples, dtype=int), np.ones_like(onset_samples, dtype=int)])

    # 4) prepare reject dict for MNE (if provided)
    reject = {'eeg': float(reject_threshold)} if (reject_threshold is not None) else None

    # default baseline: keep original behavior (if baseline is None, we set (tmin, 0) to mimic prior code)
    if baseline is None:
        baseline = (tmin, 0)

    # 5) create epochs
    epochs = mne.Epochs(
        raw, mne_events, event_id={stim_label: 1},
        tmin=tmin, tmax=tmax, baseline=baseline,
        detrend=detrend, preload=True, picks=picks, reject=reject, verbose=False
    )

    X = epochs.get_data()   # shape (n_trials, n_channels, n_times)
    n_original = len(stim_events)
    n_kept = X.shape[0]
    n_dropped = n_original - n_kept

    # simple info print (you can replace with logging)
    print(f"[epoch_sus_trials] subject={os.path.basename(eeg_file).split('_')[0]} | requested {n_original} events -> kept {n_kept} epochs (dropped {n_dropped} by reject).")

    # 6) grouping (optional)
    grouped = None
    if group_size is not None:
        if n_kept < group_size:
            print(f"[epoch_sus_trials] Warning: kept epochs ({n_kept}) < group_size ({group_size}) -> no groups produced.")
            grouped = np.empty((0,) + X.shape[1:], dtype=X.dtype)
        else:
            idx = np.arange(n_kept)
            if not allow_overlap:
                rng.shuffle(idx)
                n_groups = n_kept // group_size
                grouped_list = []
                for i in range(n_groups):
                    sel = idx[i*group_size:(i+1)*group_size]
                    grouped_list.append(X[sel].mean(axis=0))
                grouped = np.stack(grouped_list, axis=0) if grouped_list else np.empty((0,) + X.shape[1:], dtype=X.dtype)
            else:
                # overlapping / with replacement groups
                n_groups = max(1, n_kept // group_size)
                grouped_list = []
                for _ in range(n_groups):
                    sel = rng.choice(idx, size=group_size, replace=True)
                    grouped_list.append(X[sel].mean(axis=0))
                grouped = np.stack(grouped_list, axis=0)

    subj = os.path.basename(eeg_file).split("_")[0]

    # return flexible
    if group_size is None:
        if return_original_epochs:
            return X, None, subj
        else:
            return X, subj
    else:
        if return_original_epochs:
            return X, grouped, subj
        else:
            return grouped, subj

In [50]:
def make_grouped_erp(epoch_list, g_erp=3, random_state=None, allow_overlap=False):
    """Group epochs into non-overlapping averaged ERPs.
       epoch_list: list of arrays (C x T) or flattened vectors
       allow_overlap: if True, use sampling with replacement to create more groups
    """
    rng = np.random.default_rng(random_state)
    n = len(epoch_list)
    if n < g_erp:
        return []  # not enough trials for one group
    idx = np.arange(n)
    if not allow_overlap:
        rng.shuffle(idx)
        n_groups = n // g_erp
        groups = []
        for i in range(n_groups):
            sel = idx[i*g_erp:(i+1)*g_erp]
            grp = np.mean([epoch_list[j] for j in sel], axis=0)
            groups.append(grp)
        return groups
    else:
        # allow_overlap: sample groups with replacement
        groups = []
        n_groups = max(1, n // g_erp)
        for _ in range(n_groups):
            sel = rng.choice(idx, size=g_erp, replace=True)
            grp = np.mean([epoch_list[j] for j in sel], axis=0)
            groups.append(grp)
        return groups

### CCD RT

In [51]:
def parse_ccd_events(events_tsv, subject_from_filename=True):
    """
    Reads an events.tsv file and extracts trial-level reaction times (RTs)
    for the Contrast Change Detection (CCD) task.
    It only considers button presses that occur between a target onset
    and the subsequent contrastTrial_start.

    Args:
        events_tsv (str): Path to the events.tsv file.
        subject_from_filename (bool): If True, also return subject ID
                                      extracted from filename prefix (e.g., sub-001).

    Returns:
        np.ndarray: 1D array of RTs (in seconds).
        str (optional): subject ID if subject_from_filename=True
    """
    df = pd.read_csv(events_tsv, sep="\t")
    trials = []

    # find targets and trial start markers
    targets = df[df["value"].isin(["left_target", "right_target"])].reset_index(drop=True)
    contrast_starts = df[df["value"] == "contrastTrial_start"].reset_index(drop=True)

    for _, target in targets.iterrows():
        t_start = target["onset"]

        # next contrastTrial_start onset (trial boundary)
        next_trial_onsets = contrast_starts[contrast_starts["onset"] > t_start]["onset"]
        if not next_trial_onsets.empty:
            t_end_of_trial = next_trial_onsets.iloc[0]
        else:
            t_end_of_trial = np.inf  # no next trial → until end of file

        # button presses in between
        presses = df[(df["onset"] > t_start) &
                     (df["onset"] < t_end_of_trial) &
                     (df["value"].str.contains("buttonPress"))]

        if presses.empty:
            continue

        # first press RT
        first_press = presses.iloc[0]
        rt = first_press["onset"] - t_start
        trials.append(rt)

    rts = np.array(trials, dtype=float)

    if subject_from_filename:
        subj = os.path.basename(events_tsv).split("_")[0]  # e.g., sub-000
        return rts, subj
    else:
        return rts

### Matching

In [52]:
def build_matched_data(sus_files, ccd_files, g_erp=3, random_state=0, allow_overlap=False):
    """
    SuS ERP 그룹과 CCD trial-level RT를 subject 내에서 1:1 매칭하는 함수.

    Args:
        sus_files (list): list of (eeg_file, events_file) for SuS
        ccd_files (list): list of events.tsv files for CCD
        g_erp (int): SuS ERP를 만들 때 몇 trial을 평균할지
        random_state (int): 랜덤 시드
        allow_overlap (bool): ERP 그룹 만들 때 trial overlap 허용 여부

    Returns:
        X_all (np.ndarray): (n_samples, C, T) ERP 샘플
        y_all (np.ndarray): (n_samples,) 대응하는 RT
        subj_ids (np.ndarray): (n_samples,) subject id
    """
    rng = np.random.default_rng(random_state)

    subj_erp = {}
    subj_rt = {}

    # SuS ERP 그룹화
    for eeg_file, events_file in sus_files:
        X, subj = epoch_sus_trials(eeg_file, events_file)  # (n_trials, C, T)
        erp_groups = make_grouped_erp(list(X), g_erp=g_erp,
                                      random_state=rng, allow_overlap=allow_overlap)
        if subj in subj_erp:
            subj_erp[subj].extend(erp_groups)
        else:
            subj_erp[subj] = erp_groups

    # CCD RT (trial-level)
    for ev_file in ccd_files:
        rts, subj = parse_ccd_events(ev_file)  # np.array
        if len(rts) > 0:
            if subj in subj_rt:
                subj_rt[subj] = np.concatenate([subj_rt[subj], rts])
            else:
                subj_rt[subj] = rts

    # 공통 subject만 추출 & 매칭
    common_subj = set(subj_erp.keys()) & set(subj_rt.keys())
    X_all, y_all, subj_ids = [], [], []

    for subj in common_subj:
        erps = subj_erp[subj]
        rts = subj_rt[subj]

        n_pairs = min(len(erps), len(rts))
        if n_pairs == 0:
            continue

        # 랜덤 셔플 후 1:1 매칭
        erp_idx = np.arange(len(erps))
        rt_idx = np.arange(len(rts))
        rng.shuffle(erp_idx)
        rng.shuffle(rt_idx)

        for i in range(n_pairs):
            X_all.append(erps[erp_idx[i]])
            y_all.append(rts[rt_idx[i]])
            subj_ids.append(subj)

    return np.array(X_all), np.array(y_all), np.array(subj_ids)


### Meta 데이터 불러오기

In [53]:
def load_all_participants(bids_root, datasets):
    """
    모든 release 폴더(ds005505 ~ ds005516)의 participants.tsv를 읽고
    subject ID 기준으로 통합 dataframe 생성
    """
    dfs = []
    for ds in datasets:
        pfile = os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df = pd.read_csv(pfile, sep="\t")
            dfs.append(df)
    df_all = pd.concat(dfs, ignore_index=True)
    df_all = df_all.drop_duplicates(subset=["participant_id"])
    df_all = df_all.set_index("participant_id")
    return df_all


In [54]:
def extract_demographic_meta(df_all, subj_ids):
    meta_features = []
    for subj in subj_ids:
        if subj in df_all.index:
            row = df_all.loc[subj]
            # NaN-safe 변환 (없으면 0으로 대체)
            age = float(row["age"]) if pd.notna(row["age"]) else 0.0
            sex = 1 if row["sex"] == "M" else 0
            ehq = float(row["ehq_total"]) if pd.notna(row["ehq_total"]) else 0.0
            meta_features.append([age, sex, ehq])
        else:
            meta_features.append([0.0, 0.0, 0.0])  # 없는 경우 기본값
    return np.array(meta_features, dtype=np.float32)


## Dataset 정의

In [55]:
class SubjectDataset(Dataset):
    def __init__(self, X, meta, y, subj_ids):
        self.X = torch.tensor(X, dtype=torch.float32)            # (N, C, T)
        self.meta = torch.tensor(meta, dtype=torch.float32)      # (N, 3)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)
        self.subj_ids = subj_ids

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return {
            "eeg": self.X[idx],
            "meta": self.meta[idx],
            "label": self.y[idx],
            "subject": self.subj_ids[idx]
        }


## 모델 정의

In [56]:
import torch
import torch.nn as nn

class EEGNeX_Custom(nn.Module):
    def __init__(self, n_chans, n_times, d_meta=3, feat_dim=128, dropout_p=0.3):
        super().__init__()
        # Temporal conv
        self.temporal = nn.Conv1d(n_chans, 64, kernel_size=15, padding=7)
        self.bn1 = nn.BatchNorm1d(64)

        # Depthwise + pointwise
        self.depthwise = nn.Conv1d(64, 64, kernel_size=15, padding=7, groups=64)
        self.pointwise = nn.Conv1d(64, 128, kernel_size=1)

        # Residual
        self.residual = nn.Conv1d(n_chans, 128, kernel_size=1)

        # Channel attention (SE block)
        self.se_fc1 = nn.Linear(128, 32)
        self.se_fc2 = nn.Linear(32, 128)

        # Pooling
        self.pool = nn.AdaptiveAvgPool1d(50)

        # EEG feature extractor
        self.fc_eeg = nn.Sequential(
            nn.Linear(128*50, feat_dim),
            nn.ReLU(),
            nn.Dropout(dropout_p)   # Dropout 추가
        )

        # Meta branch
        self.fc_meta = nn.Sequential(
            nn.Linear(d_meta, 16),
            nn.ReLU(),
            nn.Dropout(dropout_p),  # Dropout 추가
            nn.Linear(16, 8),
            nn.ReLU()
        )

        # Fusion
        self.dropout_fusion = nn.Dropout(dropout_p)
        self.fc_fusion = nn.Linear(feat_dim + 8, 1)

    def forward(self, x, meta):
        # EEG branch
        out = torch.relu(self.bn1(self.temporal(x)))   # (B, 64, T)
        out = self.pointwise(self.depthwise(out))      # (B, 128, T)

        # Residual
        res = self.residual(x)                         # (B, 128, T)
        out = out + res

        # Channel attention
        w = torch.mean(out, dim=2)                     # (B, 128)
        w = torch.relu(self.se_fc1(w))
        w = torch.sigmoid(self.se_fc2(w)).unsqueeze(2) # (B, 128, 1)
        out = out * w

        # Pooling & flatten
        out = self.pool(out)                           # (B, 128, 50)
        out = out.flatten(start_dim=1)                 # (B, 128*50)

        eeg_feat = self.fc_eeg(out)                    # (B, feat_dim)
        meta_feat = self.fc_meta(meta)                 # (B, 8)

        # Fusion
        fused = torch.cat([eeg_feat, meta_feat], dim=1)
        fused = self.dropout_fusion(fused)             # Dropout 추가
        rt = self.fc_fusion(fused)                     # (B, 1)

        return rt


## 학습 루프

In [57]:
def train_model(train_loader, val_loader, n_chans, n_times, d_meta=3, n_epochs=20, lr=1e-3, device="cuda"):
    model = EEGNeX_Custom(n_chans=n_chans, n_times=n_times, d_meta=d_meta, dropout_p=0.3).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    criterion = nn.MSELoss()

    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
            x, m, y = batch["eeg"].to(device), batch["meta"].to(device), batch["label"].to(device)
            optimizer.zero_grad()
            y_pred = model(x, m)
            loss = criterion(y_pred, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
            optimizer.step()
            train_loss += loss.item() * len(x)
        train_loss /= len(train_loader.dataset)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                x, m, y = batch["eeg"].to(device), batch["meta"].to(device), batch["label"].to(device)
                y_pred = model(x, m)
                loss = criterion(y_pred, y)
                val_loss += loss.item() * len(x)
        val_loss /= len(val_loader.dataset)

        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}")
    return model


In [69]:
# ds005505 ~ ds005516
all_datasets = [f"ds00{i}" for i in range(5505, 5517)]

# Train: Release5(ds005509) 제외
train_datasets = [ds for ds in all_datasets if ds != "ds005509"]
val_datasets   = ["ds005509"]

# 기본 경로 설정
BIDS_ROOT = "/data5/open_data/HBN"

# events.tsv 파일의 새로운 경로
BIDS_EEG_ROOT = os.path.join(BIDS_ROOT, "EEG_BIDS")

# eeg.set 파일의 경로 (이전과 동일)
PREPROCESSED_EEG_ROOT = os.path.join(BIDS_ROOT, "Preprocessed_EEG", "0922try_bySubject")

def collect_sus_files(datasets):
    all_files = []
    
    for ds in datasets:
        # events.tsv 파일은 BIDS_EEG_ROOT 내의 'ds' 폴더에서 찾습니다.
        events_root = os.path.join(BIDS_EEG_ROOT, ds)
        
        # 1. 원본 경로에서 'events.tsv' 파일을 찾습니다.
        events_files = glob(os.path.join(events_root, "sub-*", "eeg", f"sub-*_task-surroundSupp_run-*_events.tsv"))
        
        # 2. 찾은 events 파일을 기반으로 대응하는 전처리된 eeg 파일을 연결합니다.
        for events_file in events_files:
            # 파일 경로에서 subject ID를 추출합니다.
            basename = os.path.basename(events_file).replace("_events.tsv", "")
            subject_id = basename.split('_')[0]
            
            # 3. eeg.set 파일의 경로는 PREPROCESSED_EEG_ROOT에서 찾습니다.
            preprocessed_eeg_file_path = os.path.join(PREPROCESSED_EEG_ROOT, subject_id, f"{basename}_eeg_pp.set")
            
            # 파일이 존재하는지 확인하고 쌍을 묶습니다.
            if os.path.exists(preprocessed_eeg_file_path):
                all_files.append((preprocessed_eeg_file_path, events_file))
            else:
                print(f"Warning: Corresponding eeg.set not found for {events_file}")
                
    return all_files

def collect_ccd_files(datasets):
    all_files = []
    for ds in datasets:
        bids_root = os.path.join(BIDS_EEG_ROOT, ds)
        ev_files = glob(os.path.join(bids_root, "sub-*", "eeg", f"sub-*_task-contrastChangeDetection_run-*_events.tsv"))
        all_files.extend(ev_files)
    return all_files

BIDS_ROOT_2 = "/data5/open_data/HBN/EEG_BIDS/"

# 1. participants.tsv 병합
df_participants = load_all_participants(BIDS_ROOT_2, train_datasets)

# 2. SuS & CCD 파일 수집
sus_train_files = collect_sus_files(train_datasets)
ccd_train_files = collect_ccd_files(train_datasets)
sus_val_files   = collect_sus_files(val_datasets)
ccd_val_files   = collect_ccd_files(val_datasets)

일부 데이터만 사용해서 오류 체크

In [14]:
# ds005505 ~ ds005516
all_datasets = ["ds005505", "ds005509"]

# Train: Release5(ds005509) 제외
train_datasets = ["ds005505"]
val_datasets   = ["ds005509"]

BIDS_ROOT = "/data5/open_data/HBN/EEG_BIDS"

def collect_sus_files(datasets):
    all_files = []
    for ds in datasets:
        bids_root = os.path.join(BIDS_ROOT, ds)
        eeg_files = glob(os.path.join(bids_root, "sub-*", "eeg", f"sub-*_task-surroundSupp_run-*_eeg.set"))
        for eeg_file in eeg_files:
            events_file = eeg_file.replace("_eeg.set", "_events.tsv")
            all_files.append((eeg_file, events_file))
    return all_files

def collect_ccd_files(datasets):
    all_files = []
    for ds in datasets:
        bids_root = os.path.join(BIDS_ROOT, ds)
        ev_files = glob(os.path.join(bids_root, "sub-*", "eeg", f"sub-*_task-contrastChangeDetection_run-*_events.tsv"))
        all_files.extend(ev_files)
    return all_files



# 1. participants.tsv 병합
df_participants = load_all_participants(BIDS_ROOT, train_datasets)

# 2. SuS & CCD 파일 수집
sus_train_files = collect_sus_files(train_datasets)
ccd_train_files = collect_ccd_files(train_datasets)
sus_val_files   = collect_sus_files(val_datasets)
ccd_val_files   = collect_ccd_files(val_datasets)

In [68]:
print(len(sus_val_files))

549


In [70]:
# 3. subject-level ERP + RT
X_train, y_train, subj_train = build_matched_data(sus_train_files, ccd_train_files, g_erp=3, random_state=42)
X_val, y_val, subj_val       = build_matched_data(sus_val_files, ccd_val_files, g_erp=3, random_state=42)

[epoch_sus_trials] subject=sub-NDARZD415ZZ1 | requested 64 events -> kept 4 epochs (dropped 60 by reject).


/tmp/ipykernel_2741048/1107793769.py:25: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_file, preload=True, verbose=False)


[epoch_sus_trials] subject=sub-NDARZD415ZZ1 | requested 64 events -> kept 31 epochs (dropped 33 by reject).


/tmp/ipykernel_2741048/1107793769.py:25: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_file, preload=True, verbose=False)


[epoch_sus_trials] subject=sub-NDARXH597ML1 | requested 64 events -> kept 49 epochs (dropped 15 by reject).
[epoch_sus_trials] subject=sub-NDARVD685RRJ | requested 64 events -> kept 58 epochs (dropped 6 by reject).


...

[epoch_sus_trials] subject=sub-NDARZJ587UZU | requested 64 events -> kept 37 epochs (dropped 27 by reject).
[epoch_sus_trials] subject=sub-NDARZJ587UZU | requested 64 events -> kept 39 epochs (dropped 25 by reject).
[epoch_sus_trials] subject=sub-NDARZM903TNL | requested 64 events -> kept 23 epochs (dropped 41 by reject).
[epoch_sus_trials] subject=sub-NDARZM903TNL | requested 64 events -> kept 49 epochs (dropped 15 by reject).
[epoch_sus_trials] subject=sub-NDARZN088VER | requested 64 events -> kept 7 epochs (dropped 57 by reject).
[epoch_sus_trials] subject=sub-NDARZW472CCF | requested 64 events -> kept 28 epochs (dropped 36 by reject).


/tmp/ipykernel_2741048/1107793769.py:25: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_file, preload=True, verbose=False)
/tmp/ipykernel_2741048/1107793769.py:92: RuntimeWarning: All epochs were dropped!
You might need to alter reject/flat-criteria or drop bad channels to avoid this. You can use Epochs.plot_drop_log() to see which channels are responsible for the dropping of epochs.
  epochs = mne.Epochs(
/tmp/ipykernel_2741048/1107793769.py:98: RuntimeWarning: epochs._get_data() can't run because this Epochs-object is empty. You might want to check Epochs.drop_log or Epochs.plot_drop_log() to see why epochs were dropped.
  X = epochs.get_data()   # shape (n_trials, n_channels, n_times)


[epoch_sus_trials] subject=sub-NDARZW472CCF | requested 64 events -> kept 0 epochs (dropped 64 by reject).
[epoch_sus_trials] subject=sub-NDARZZ740MLM | requested 64 events -> kept 24 epochs (dropped 40 by reject).


/tmp/ipykernel_2741048/1107793769.py:25: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_file, preload=True, verbose=False)


[epoch_sus_trials] subject=sub-NDARZZ740MLM | requested 64 events -> kept 23 epochs (dropped 41 by reject).


In [71]:
import numpy as np

# Train set 저장
np.savez_compressed(
    "ch1_2_train_subject_level.npz",
    X=X_train, 
    y=y_train, 
    subj=subj_train
)

# Val set 저장
np.savez_compressed(
    "ch1_2_ccd_val_subject_level.npz",
    X=X_val, 
    y=y_val, 
    subj=subj_val
)


In [27]:
len(X_train)

346

In [72]:
# 4. meta 추출
meta_train = extract_demographic_meta(df_participants, subj_train)
meta_val   = extract_demographic_meta(df_participants, subj_val)

In [73]:
from sklearn.preprocessing import StandardScaler

# meta_train, meta_val 준비된 상태라고 가정
scaler = StandardScaler()

# Train set 기준으로 fit → transform
meta_train_scaled = scaler.fit_transform(meta_train)
meta_val_scaled   = scaler.transform(meta_val)


In [24]:
meta_train[0]

array([ 7.3331,  1.    , 80.04  ], dtype=float32)

In [74]:
# 5. Dataset & Loader
train_dataset = SubjectDataset(X_train, meta_train_scaled, y_train, subj_train)
val_dataset   = SubjectDataset(X_val, meta_val_scaled, y_val, subj_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
sample = train_dataset[0]
print("Keys:", sample.keys())
print("EEG shape:", sample["eeg"].shape)   # (C, T)
print("Meta shape:", sample["meta"].shape) # (3,)
print("Label:", sample["label"].item())    # scalar
print("Subject:", sample["subject"])

Keys: dict_keys(['eeg', 'meta', 'label', 'subject'])
EEG shape: torch.Size([129, 351])
Meta shape: torch.Size([3])
Label: 1.719181776046753
Subject: sub-NDARPW313LAJ


In [75]:
n_chans = X_train.shape[1]   # EEG 채널 수 (ex: 129)
n_times = X_train.shape[2]   # ERP timepoints (ex: 351)
d_meta  = meta_train.shape[1] # 메타데이터 feature 수 (3)


model = train_model(
    train_loader, 
    val_loader,
    n_chans=n_chans, 
    n_times=n_times, 
    d_meta=d_meta, 
    n_epochs=100, 
    lr=1e-4,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

Epoch 1/100: 100%|██████████| 2221/2221 [00:14<00:00, 149.63it/s]


Epoch 1: Train Loss=0.2637, Val Loss=0.1799


Epoch 2/100: 100%|██████████| 2221/2221 [00:15<00:00, 140.82it/s]


Epoch 2: Train Loss=0.2346, Val Loss=0.2076


Epoch 3/100: 100%|██████████| 2221/2221 [00:14<00:00, 148.10it/s]


...

Epoch 98: Train Loss=0.0927, Val Loss=0.5953


Epoch 99/100: 100%|██████████| 2221/2221 [00:14<00:00, 151.04it/s]


Epoch 99: Train Loss=0.0916, Val Loss=0.6024


Epoch 100/100: 100%|██████████| 2221/2221 [00:14<00:00, 151.18it/s]


Epoch 100: Train Loss=0.0909, Val Loss=0.3949


In [46]:
# saving the model

torch.save(model.state_dict(), "ch1_1_model.pth")

각 task, subject 별로 trial이 어떻게 되는지 구해보기